In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import csv

options = Options()
# options.headless = True  # Uncomment to run in headless mode if needed
service = Service()  #

driver = webdriver.Chrome(service=service, options=options)


csv_file_path = 'products_data.csv'
fieldnames = ['URL', 'Product Title', 'Image URL', 'UPC', 'Last Unit Price', 'Stock Status']

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    try:
        
        driver.get('https://www.mxwholesale.co.uk/collections/all')

        while True:
            
            product_links = []
            product_elements = driver.find_elements(By.XPATH, '//a[contains(@href, "/products/")]')
            for index, element in enumerate(product_elements):
                if index % 2 == 0:  
                    product_links.append(element.get_attribute('href'))

            
            for href in product_links:
                print(f"Processing link: {href}")

                
                driver.get(href)

                
                html_content = driver.page_source
                soup = BeautifulSoup(html_content, 'html.parser')

               
                product_title = soup.find('h1', class_='product-single__title').get_text(strip=True) if soup.find('h1', class_='product-single__title') else 'Title not found'
                image_tag = soup.find('img', class_='lazyautosizes')
                image_url = image_tag['data-photoswipe-src'] if image_tag else 'Image URL not found'
                if image_url.startswith("//"):
                    image_url = image_url[2:]
                barcode_paragraph = soup.find('p', string=lambda x: x and 'Barcode:' in x)
                barcode = barcode_paragraph.get_text(strip=True).replace('Barcode: ', '') if barcode_paragraph else 'Barcode not found'
                td_tags = soup.find_all('td')
                last_unit_price = td_tags[-1].get_text(strip=True) if td_tags else 'Unit price not found'
                stock_status = soup.find('span', class_='stock-message').get_text(strip=True) if soup.find('span', class_='stock-message') else 'Stock status not found'

                
                print(f"Product Title: {product_title}")
                print(f"Image URL: {image_url}")
                print(f"UPC: {barcode}")
                print(f"Last Unit Price: {last_unit_price}")
                print(f"Stock Status: {stock_status}")
                print("-----")

                
                writer.writerow({
                    'URL': href,
                    'Product Title': product_title,
                    'Image URL': image_url,
                    'UPC': barcode,
                    'Last Unit Price': last_unit_price,
                    'Stock Status': stock_status
                })

                
                driver.back()

                
                time.sleep(2)

            
            try:
                load_more_button = driver.find_element(By.CLASS_NAME, 'btn-load-more')
                load_more_button.click()
                print("Clicked 'Load more products' button.")
                time.sleep(2)  
            except Exception as e:
                print("No more 'Load more products' button found or all products have been loaded.")
                break

    finally:
        
        driver.quit()


The chromedriver version (126.0.6478.126) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (127.0.6533.119); currently, chromedriver 127.0.6533.119 is recommended for chrome 127.*, so it is advised to delete the driver in PATH and retry


Processing link: https://www.mxwholesale.co.uk/collections/all/products/age-1-unisex-birthday-latex-balloons-x-6-pks-of-8-balloons-1-48
Product Title: "1" Design Latex Birthday Balloons Assorted Colours 8 Pack - Case of 6
Image URL: www.mxwholesale.co.uk/cdn/shop/products/05055579165587_A_1800x1800.jpg?v=1663782478
UPC: 05055579165587
Last Unit Price: KSh128.00
Stock Status: In stock
-----
Processing link: https://www.mxwholesale.co.uk/collections/all/products/age-16-unisex-birthday-latex-balloons-x-6-pks-of-8-balloons-1-48
Product Title: "16" Design Latex Birthday Balloons Assorted Colours 8 Pack - Case of 6
Image URL: www.mxwholesale.co.uk/cdn/shop/products/5055579165129_A_1800x1800.jpg?v=1609427101
UPC: 5055579165129
Last Unit Price: KSh128.00
Stock Status: In stock
-----
Processing link: https://www.mxwholesale.co.uk/collections/all/products/age-18-unisex-birthday-latex-balloons-x-6-pks-of-8-balloons-1-48
Product Title: "18" Design Latex Birthday Balloons Assorted Colours 8 Pack - 

KeyboardInterrupt: 